In [1]:
import ray, random, os 
from ray.air.config import ScalingConfig
import pandas as pd
ray.init(_temp_dir='/Volumes/SSD980/ray')
from ray import air, tune
from ray.tune.schedulers import PopulationBasedTraining
import argparse
from ray.tune import CLIReporter
from worker_standlone import WorkerStandAlone
from manager import Manager
from multi_agent import MultiAgent
from ray.train.rl import RLTrainer
from ray.rllib.policy.policy import Policy, PolicySpec
from ray.tune.registry import register_env
from ray.tune import TuneConfig
from ray.tune.logger import pretty_print
from ray.tune.search.bayesopt import BayesOptSearch
from ray.air.config import RunConfig, CheckpointConfig
from ray.tune.schedulers import HyperBandScheduler
from ray.rllib.algorithms.ppo import PPOConfig
from ray.tune.search.hyperopt import HyperOptSearch
from ray.tune.schedulers import ASHAScheduler
from ray.tune.logger import pretty_print

from ray.rllib.algorithms.a2c import A2CConfig

2023-09-28 16:48:18,688	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
2023-09-28 16:48:25,379	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!


In [2]:
multi_agent_env = MultiAgent()


for episode in range(1):
    obs, info = multi_agent_env.reset()
    done = {"__all__": False}
    
    print(f"Episode {episode + 1}")

    while not done["__all__"]:
        actions = {}
        
        # Collect actions for each agent
        for agent_id, agent_obs in obs.items():
            action_space = multi_agent_env.action_space[agent_id]
            action = action_space.sample()
            actions[agent_id] = action

        # Step the environment
        obs, reward, done, _, info = multi_agent_env.step(actions)

        # print(f"Actions: {actions}")
        # print(f"Observations: {obs}")
        # print(f"Rewards: {reward}")
        # print(f"Done flags: {done}")
        # print(f"Info: {info}")

    print("Episode done!")

Episode 1
=========ABT.US is done=============
day: 6304, episode: 1
Beginn_Portfolio_Value: 1000000.0
End Total Assets: 3791418.5
Total PnL: 2791424.75
=========AMGN.US is done=============
day: 6304, episode: 1
Beginn_Portfolio_Value: 1000000.0
End Total Assets: 11508680.0
Total PnL: 10508690.0
=========BDX.US is done=============
day: 6304, episode: 1
Beginn_Portfolio_Value: 1000000.0
End Total Assets: 4566702.5
Total PnL: 3566699.5
=========BMY.US is done=============
day: 6304, episode: 1
Beginn_Portfolio_Value: 1000000.0
End Total Assets: 2617018.0
Total PnL: 1617016.375
=========HUM.US is done=============
day: 6304, episode: 1
Beginn_Portfolio_Value: 1000000.0
End Total Assets: 415391.9375
Total PnL: -584607.875
=========JNJ.US is done=============
day: 6304, episode: 1
Beginn_Portfolio_Value: 1000000.0
End Total Assets: 1232315.375
Total PnL: 232315.125
=========LLY.US is done=============
day: 6304, episode: 1
Beginn_Portfolio_Value: 1000000.0
End Total Assets: 1349690.75
Tot

In [2]:
env = WorkerStandAlone()
def env_creator(env_config):
    return WorkerStandAlone()  

register_env("worker", env_creator)


config = (A2CConfig().environment("worker", env_config={"tic": "value"}).rollouts(num_rollout_workers=2,create_env_on_local_worker=True))

algo = config.build()
for i in range(1):
    result = algo.train()
print(pretty_print(result))

2023-09-28 14:55:25,243	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/a2c/` has been deprecated. Use `rllib_contrib/a2c/` instead. This will raise an error in the future!
2023-09-28 14:55:25,244	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/a3c/` has been deprecated. Use `rllib_contrib/a3c/` instead. This will raise an error in the future!


tic: None


/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/ray/rllib/algorithms/algorithm.py:484: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/ray/tune/logger/unified.py:53: RayDeprec

(RolloutWorker pid=50030) tic: None
tic: None
agent_timesteps_total: 13920
connector_metrics:
  ObsPreprocessorConnector_ms: 0.023031234741210938
  StateBufferConnector_ms: 0.0019431114196777344
  ViewRequirementAgentConnector_ms: 0.05081892013549805
counters:
  num_agent_steps_sampled: 13920
  num_agent_steps_trained: 13920
  num_env_steps_sampled: 13920
  num_env_steps_trained: 13920
custom_metrics: {}
date: 2023-09-28_14-55-41
done: false
episode_len_mean: 6305.0
episode_media: {}
episode_reward_max: 13813.04180417792
episode_reward_mean: 3335.5682694841526
episode_reward_min: -7141.905265209614
episodes_this_iter: 2
episodes_total: 2
hostname: Florians-MacBook-Pro.local
info:
  learner:
    default_policy:
      custom_metrics: {}
      diff_num_grad_updates_vs_sampler_policy: 434.0
      learner_stats:
        allreduce_latency: 0.0
        cur_lr: 0.0001
        entropy_coeff: 0.01
        grad_gnorm: 40.0
        policy_entropy: 83.27497100830078
        policy_loss: 33010.0
   

In [6]:
checkpoint = algo.save()
print(checkpoint)

/Users/floriankockler/ray_results/A2C_worker_2023-09-28_14-16-434npw5fzj/checkpoint_000010


In [7]:
evaluation = algo.evaluate()
print(pretty_print(evaluation))

evaluation:
  connector_metrics: {}
  custom_metrics: {}
  episode_len_mean: .nan
  episode_media: {}
  episode_reward_max: .nan
  episode_reward_mean: .nan
  episode_reward_min: .nan
  episodes_this_iter: 0
  hist_stats:
    episode_lengths: []
    episode_reward: []
  num_agent_steps_sampled_this_iter: 160
  num_env_steps_sampled_this_iter: 160
  num_faulty_episodes: 0
  policy_reward_max: {}
  policy_reward_mean: {}
  policy_reward_min: {}
  sampler_perf: {}
  sampler_results:
    connector_metrics: {}
    custom_metrics: {}
    episode_len_mean: .nan
    episode_media: {}
    episode_reward_max: .nan
    episode_reward_mean: .nan
    episode_reward_min: .nan
    episodes_this_iter: 0
    hist_stats:
      episode_lengths: []
      episode_reward: []
    num_faulty_episodes: 0
    policy_reward_max: {}
    policy_reward_mean: {}
    policy_reward_min: {}
    sampler_perf: {}
  timesteps_this_iter: 160



In [8]:
from ray.rllib.algorithms.algorithm import Algorithm
restored_algo = Algorithm.from_checkpoint(checkpoint)

/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/ray/rllib/algorithms/algorithm.py:484: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/ray/tune/logger/unified.py:53: RayDeprec

In [10]:
policy = algo.get_policy()
print(policy.get_weights())

{'_logits._model.0.weight': array([[-8.86500347e-03, -6.32040948e-03,  6.06775889e-03, ...,
        -6.63008401e-03, -6.10738900e-03,  8.72934610e-03],
       [-5.77917416e-03,  1.93063158e-03,  1.31427741e-03, ...,
         2.90287822e-03,  9.28564987e-05,  6.29687274e-04],
       [-7.53543852e-03, -1.17568783e-02,  1.21536255e-02, ...,
        -1.70481447e-02, -1.44792125e-02,  1.37761096e-02],
       [-8.88302282e-04,  5.69259794e-03, -7.15491944e-04, ...,
         1.09620383e-02,  5.22674015e-03, -2.09682924e-03],
       [ 1.29465600e-02,  1.23090688e-02, -1.80436447e-02, ...,
         1.41422050e-02,  1.82701256e-02, -1.83018651e-02]], dtype=float32), '_logits._model.0.bias': array([ 0.00702241,  0.00213337,  0.01298092, -0.00174136, -0.01727833],
      dtype=float32), '_hidden_layers.0._model.0.weight': array([[ 0.08654266, -0.20018442,  0.3109591 , ..., -0.15258518,
        -0.16027088,  0.13268723],
       [-0.31570196,  0.5250863 ,  0.05066016, ..., -0.1441891 ,
        -0.097

AttributeError: 'FullyConnectedNetwork' object has no attribute 'base_model'